In [3]:
import sys,os,re
import pysubs2
import cv2
import threading

打印机效果：
每个字间隔两帧🎞，大约 33ms
{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(33,34,\1a&H00&\3a&H00&\4a&H00&)}

In [19]:
subs = pysubs2.load("小夏4.ass")
for i in range(5):
    if subs[i].style == "文本":
        text = subs[i].text.split("：")[-1]
        print(text)

（如夏所说今天的天空很清澄，是个天气非常好的一天）
（阳光非常暖和而不让人感到闷热，风也温和地吹拂着）
啊，老师。这里这里——
这么好的天气，抢位置的人肯定很多。


In [18]:
subs = pysubs2.load("小夏4.ass")
for i in range(5):
    if subs[i].style == "文本":
        text = sub.text.split("：")[-1]
        cnt = 1
        s = ""
        for word in text:
            s = s+"{\\1a&HFF&\\3a&HFF&\\4a&HFF&}{\\t("+str(cnt*33)+","+str(cnt*33+1)+",\\1a&H00&\\3a&H00&\\4a&H00&)}"+word
            if cnt % 35 == 0:
                s += "\\N"
            cnt+=1
        print(s)

{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(33,34,\1a&H00&\3a&H00&\4a&H00&)}（{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(66,67,\1a&H00&\3a&H00&\4a&H00&)}之{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(99,100,\1a&H00&\3a&H00&\4a&H00&)}后{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(132,133,\1a&H00&\3a&H00&\4a&H00&)}和{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(165,166,\1a&H00&\3a&H00&\4a&H00&)}夏{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(198,199,\1a&H00&\3a&H00&\4a&H00&)}一{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(231,232,\1a&H00&\3a&H00&\4a&H00&)}起{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(264,265,\1a&H00&\3a&H00&\4a&H00&)}各{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(297,298,\1a&H00&\3a&H00&\4a&H00&)}种{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(330,331,\1a&H00&\3a&H00&\4a&H00&)}享{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(363,364,\1a&H00&\3a&H00&\4a&H00&)}乐{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(396,397,\1a&H00&\3a&H00&\4a&H00&)}了{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(429,430,\1a&H00&\3a&H00&\4a&H00&)}之{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(462,463,\1a&H00&\3a&H00&\4a&H00&)}后{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(495,496,\1a&H00&\3a&H00&\4a&H00&)}，{\1a&HFF&\3a&HFF&\4a&HFF&}{\t(

In [41]:
def work(start):
    video_path = os.path.join("./temp/", "part_"+str(start)+".mp4")
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width = int(cap.get(3))
    height = int(cap.get(4))
    out = cv2.VideoWriter("./temp/part_"+str(start)+"_out.mp4",fourcc,fps,(width,height))
    print("start threading "+str(start))

    for i in range(start*num,(start+1)*num):
        ret, img = cap.read()
        if ret:
            for style in frame[i]:
                y1 ,x1 ,y2 ,x2= axis[style][0], axis[style][1], axis[style][2], axis[style][3]
                Inpainting(img,x1,y1,x2,y2,style,kernelSize=9,iter=1,r=4)
            out.write(img)
        else:
            break
        if cv2.waitKey(1) & 0xFF==27:
            break
    cap.release()
    out.release()
    print("finish threading "+str(start))

In [42]:
num = 1000
def Inpainting(srcImg,x,y,xx,yy,style="文本",kernelSize=7,iter=1,r=3):
    src = srcImg[x:xx,y:yy].copy()
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY,1)
    if "fadeout" in style:
        thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    elif style == "学生" or style == "文本": #白字
        thresh = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)[1]
    else:
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernelSize, kernelSize))
    maskImg = cv2.dilate(thresh, kernel,iterations=iter)
    inpaintImg = cv2.inpaint(src,maskImg,r,cv2.INPAINT_NS)
    srcImg[x:xx,y:yy] = inpaintImg

In [43]:
work(7)

start threading 7
finish threading 7
